In [43]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

import pandas as pd
from pattern.en import parse
from pattern.en import pprint
from pattern.vector import stem, PORTER, LEMMA

In [74]:
df = pd.read_csv('comments2.csv')
df = df.dropna()

In [45]:
import os
#os.environ['PYSPARK_PYTHON'] = '/anaconda/bin/python'
import findspark
findspark.init()
print findspark.find()
# Depending on your setup you might have to change this line of code
#findspark makes sure I dont need the below on homebrew.
os.environ['SPARK_HOME']="/usr/local/Cellar/apache-spark/1.5.1/libexec/"
#the below actually broke my spark, so I removed it. 
#Depending on how you started the notebook, you might need it.
#os.environ['PYSPARK_SUBMIT_ARGS']="--master local pyspark --executor-memory 4g"

/usr/local/opt/apache-spark/libexec


In [46]:
import pyspark
conf = (pyspark.SparkConf()
    .setMaster('local')
    .setAppName('pyspark')
    .set("spark.executor.memory", "2g"))
sc = pyspark.SparkContext(conf=conf)
import sys
rdd = sc.parallelize(xrange(10),10)
rdd.map(lambda x: sys.version).collect()

['2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x86_64)| (default, Oct 19 2015, 18:31:17) \n[GCC 4.2.1 (Apple Inc. build 5577)]',
 '2.7.10 |Anaconda 2.3.0 (x8

In [75]:
from pyspark.sql import SQLContext
sqlsc=SQLContext(sc)
s_df = sqlsc.createDataFrame(df)

In [76]:
s_df.show(2)

+----------+--------------------+-------+------------+--------------------+-----+--------------------+
|Unnamed: 0|                body|company|        date|                link|score|               title|
+----------+--------------------+-------+------------+--------------------+-----+--------------------+
|         1|I love how compan...| Google|1428937417.0|https://www.reddi...|  429|1089 :: Google Fi...|
|         3|I should've known...| Google|1402464010.0|https://www.reddi...|   79|831 :: Google jus...|
+----------+--------------------+-------+------------+--------------------+-----+--------------------+
only showing top 2 rows



In [20]:
#df = df.drop('Unnamed: 0', axis=1)

In [25]:
import re
regex1=re.compile(r"\.{2,}")
regex2=re.compile(r"\-{2,}")

In [39]:
from pattern.vector import stem, PORTER, LEMMA
punctuation = list('.,;:!?()[]{}`''\"@#$^&*+-|=~_')
from sklearn.feature_extraction import text 
stopwords=text.ENGLISH_STOP_WORDS

In [40]:
def get_parts(thetext):
    thetext=re.sub(regex1, ' ', thetext)
    thetext=re.sub(regex2, ' ', thetext)
    nouns=[]
    descriptives=[]
    for i,sentence in enumerate(parse(thetext, tokenize=True, lemmata=True).split()):
        nouns.append([])
        descriptives.append([])
        for token in sentence:
            #print token
            if len(token[4]) >0:
                if token[1] in ['JJ', 'JJR', 'JJS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    descriptives[i].append(token[4])
                elif token[1] in ['NN', 'NNS']:
                    if token[4] in stopwords or token[4][0] in punctuation or token[4][-1] in punctuation or len(token[4])==1:
                        continue
                    nouns[i].append(token[4])
    out=zip(nouns, descriptives)
    nouns2=[]
    descriptives2=[]
    for n,d in out:
        if len(n)!=0 and len(d)!=0:
            nouns2.append(n)
            descriptives2.append(d)
    return nouns2, descriptives2

In [77]:
# for i in df.body:
#     print get_parts(i)
#     print 'hi'
#     break
review_parts = s_df.map(lambda r: get_parts(r.body))

In [110]:
review_parts.take(1)


[([[u'company', u'speed', u'pricing', u'roll', u'speed'],
   [u'type',
    u'paper',
    u'towel',
    u'stock',
    u'truck-load',
    u'supply',
    u'paper',
    u'towel',
    u'warehouse'],
   [u'power', u'quality', u'internet', u'service', u'price', u'competition'],
   [u'industry'],
   [u'way', u'country']],
  [[u'competitive', u'sudden'],
   [u'certain', u'crappy'],
   [u'moderate'],
   [u'standardized'],
   [u'21st']])]

In [80]:
%%time
parseout=review_parts.collect()

CPU times: user 232 ms, sys: 20.6 ms, total: 252 ms
Wall time: 2min 9s


In [81]:
[e[0] for e in parseout[:3]]

[[[u'company', u'speed', u'pricing', u'roll', u'speed'],
  [u'type',
   u'paper',
   u'towel',
   u'stock',
   u'truck-load',
   u'supply',
   u'paper',
   u'towel',
   u'warehouse'],
  [u'power', u'quality', u'internet', u'service', u'price', u'competition'],
  [u'industry'],
  [u'way', u'country']],
 [[u'month'],
  [u'government', u'operation', u'person', u'vehicle', u'acronym', u'tech']],
 []]

In [88]:
nbdatardd=sc.parallelize([ele[1] for ele in parseout])
nbdatardd.cache()
nbdatardd.take(3)

[[[u'competitive', u'sudden'],
  [u'certain', u'crappy'],
  [u'moderate'],
  [u'standardized'],
  [u'21st']],
 [[u'video'], [u'realtime']],
 []]

In [89]:
#your code here
adjvocab = (nbdatardd.flatMap(lambda word: word)
             .flatMap(lambda word:word)
             .map(lambda word: (word, 1))
             .reduceByKey(lambda a, b: a + b)
             .map(lambda (x,y): x)
             .zipWithIndex()
).cache()

In [98]:
resparray = s_df.map(lambda r: r.body).collect()

adjvocab = adjvocab.collectAsMap()
adjvocab['delicious']

In [92]:
import itertools
Xarraypre=nbdatardd.map(lambda l: " ".join(list(itertools.chain.from_iterable(l))))
Xarray=Xarraypre.collect()

In [94]:
print Xarray[0]

 competitive sudden certain crappy moderate standardized 21st


In [95]:
from sklearn.cross_validation import train_test_split
itrain, itest = train_test_split(xrange(len(Xarray)), train_size=0.7)
mask=np.ones(len(Xarray), dtype='int')
mask[itrain]=1
mask[itest]=0
mask = (mask==1)

In [99]:
X=np.array(Xarray)
y=np.array(resparray)

In [108]:
def make_xy(X_col, y_col, vectorizer):
    X = vectorizer.fit_transform(X_col)
    y = y_col
    return X, y
def log_likelihood(clf, x, y):
    prob = clf.predict_log_proba(x)
    rotten = y == 0
    fresh = ~rotten
    return prob[rotten, 0].sum() + prob[fresh, 1].sum()
def calibration_plot(clf, xtest, ytest):
    prob = clf.predict_proba(xtest)[:, 1]
    outcome = ytest
    data = pd.DataFrame(dict(prob=prob, outcome=outcome))

    #group outcomes into bins of similar probability
    bins = np.linspace(0, 1, 20)
    cuts = pd.cut(prob, bins)
    binwidth = bins[1] - bins[0]
    
    #freshness ratio and number of examples in each bin
    cal = data.groupby(cuts).outcome.agg(['mean', 'count'])
    cal['pmid'] = (bins[:-1] + bins[1:]) / 2
    cal['sig'] = np.sqrt(cal.pmid * (1 - cal.pmid) / cal['count'])
        
    #the calibration plot
    ax = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    p = plt.errorbar(cal.pmid, cal['mean'], cal['sig'])
    plt.plot(cal.pmid, cal.pmid, linestyle='--', lw=1, color='k')
    plt.ylabel("Empirical P(+)")
    
    #the distribution of P(+)
    ax = plt.subplot2grid((3, 1), (2, 0), sharex=ax)
    
    plt.bar(left=cal.pmid - binwidth / 2, height=cal['count'],
            width=.95 * (bins[1] - bins[0]),
            fc=p[0].get_color())
    
    plt.xlabel("Predicted P(+)")
    plt.ylabel("Number")
from sklearn.cross_validation import KFold

def cv_score(clf, x, y, score_func, nfold=5):
    """
    Uses 5-fold cross validation to estimate a score of a classifier
    
    Inputs
    ------
    clf : Classifier object
    x : Input feature vector
    y : Input class labels
    score_func : Function like log_likelihood, that takes (clf, x, y) as input,
                 and returns a score
                 
    Returns
    -------
    The average score obtained by splitting (x, y) into 5 folds of training and 
    test sets, fitting on the training set, and evaluating score_func on the test set
    
    Examples
    cv_score(clf, x, y, log_likelihood)
    """
    result = 0
    for train, test in KFold(y.size, nfold): # split data into train/test groups, 5 times
        clf.fit(x[train], y[train]) # fit
        result += score_func(clf, x[test], y[test]) # evaluate score function on held-out data
    return result / nfold # average
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [106]:
alphas = [0, .1, 1, 5, 10, 50]
min_dfs = [1e-5,  1e-3,  1e-1]

#Find the best value for alpha and min_df, and the best classifier
best_alpha = None
best_min_df = None
maxscore=-np.inf
      
vectorizer = CountVectorizer(adjvocab)       
Xthis, ythis = make_xy(X,y, vectorizer)
Xtrainthis=Xthis[mask]
ytrainthis=ythis[mask]
#your code here
clf = MultinomialNB(alpha=.1)
cvscore = cv_score(clf, Xtrainthis, ytrainthis, log_likelihood)

if cvscore > maxscore:
    maxscore = cvscore
    best_alpha, best_min_df = alpha, min_df

/Applications/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


In [107]:
vectorizer = CountVectorizer(vocabulary=adjvocab, min_df = best_min_df)
Xtr, ytr = make_xy(X,y, vectorizer)
xtrain=Xtr[mask]
ytrain=ytr[mask]
xtest=Xtr[~mask]
ytest=ytr[~mask]

clf = MultinomialNB(alpha=best_alpha).fit(xtrain, ytrain)

# Your code here. Print the accuracy on the test and training dataset
training_accuracy = clf.score(xtrain, ytrain)
test_accuracy = clf.score(xtest, ytest)

print "Accuracy on training data: %0.2f" % (training_accuracy)
print "Accuracy on test data:     %0.2f" % (test_accuracy)



Accuracy on training data: 0.03
Accuracy on test data:     0.03


NameError: name 'calibration_plot' is not defined

In [109]:
calibration_plot(clf, xtest, ytest)

DataError: No numeric types to aggregate